In [16]:
# Install Dependencies

In [17]:
!pip install torch_snippets torch_summary
!pip install transformers

# Import Libraries

In [18]:
import torch
from torch import nn, optim
from torchvision import transforms, models
from torch_snippets import *
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary 
from transformers import SwinForImageClassification

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import cv2
from glob import glob
import pandas as pd

sns.set_theme()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Dataset

In [19]:
DIR = "../input/human-action-recognition-har-dataset/Human Action Recognition/"
TRAIN_DIR=f"{DIR}train"
TEST_DIR=f"{DIR}test"
TRAIN_VAL_DF = "../input/human-action-recognition-har-dataset/Human Action Recognition/Training_set.csv"

In [20]:
train_val_data=glob(TRAIN_DIR+'/*.jpg')
# remove duplicate
train_val_data.remove('../input/human-action-recognition-har-dataset/Human Action Recognition/train/Image_10169(1).jpg')
train_data, val_data = train_test_split(train_val_data, test_size=0.2,
                                       shuffle=True)
print('Train Size', len(train_data))
print('Val Size', len(val_data))

In [21]:
df=pd.read_csv(f"{DIR}Training_set.csv")
df.head()

In [22]:
agg_labels = df.groupby('label').agg({'label': 'count'})
agg_labels.rename(columns={'label': 'count'})

In [23]:
ind2cat = sorted(df['label'].unique().tolist())
cat2ind = {cat: ind for ind, cat in enumerate(ind2cat)}

In [24]:
class HumanActionData(Dataset):
    def __init__(self, file_paths, df_path, cat2ind):
        super().__init__()
        self.file_paths = file_paths
        self.cat2ind = cat2ind
        self.df = pd.read_csv(df_path)
        self.transform = transforms.Compose([ 
            transforms.Resize([224, 224]), 
            transforms.ToTensor(),
            # std multiply by 255 to convert img of [0, 255]
            # to img of [0, 1]
            transforms.Normalize((0.485, 0.456, 0.406), 
                                 (0.229*255, 0.224*255, 0.225*255))]
        )
    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, ind):
        file_path = self.file_paths[ind]
        itarget = int(fname(file_path)[6:-4])
        target = self.df.iloc[itarget-1]['label']
        target = self.cat2ind[target]
        img = Image.open(file_path).convert('RGB')
        return img, target
    
    def collate_fn(self, data):
        imgs, targets = zip(*data)
        imgs = torch.stack([self.transform(img) for img in imgs], 0)
        imgs = imgs.to(device)
        targets = torch.tensor(targets).long().to(device)
        return imgs, targets
    
    def choose(self):
        return self[np.random.randint(len(self))]

In [25]:
train_ds = HumanActionData(train_data, TRAIN_VAL_DF, cat2ind)
train_dl = DataLoader(train_ds, batch_size=128, shuffle=True,
                      collate_fn=train_ds.collate_fn,
                      drop_last=True)

val_ds = HumanActionData(val_data, TRAIN_VAL_DF, cat2ind)
val_dl = DataLoader(val_ds, batch_size=128, shuffle=True,
                    collate_fn=val_ds.collate_fn,
                    drop_last=True)

In [26]:
img, target = train_ds.choose()
show(img, title=ind2cat[int(target)])

In [27]:
inspect(*next(iter(train_dl)), names='image, target')

# Model

In [36]:
class ActionClassifier_resnet152(nn.Module):
    def __init__(self, ntargets):
        super().__init__()
        resnet152 = models.resnet152(pretrained=True, progress=True)
        modules = list(resnet152.children())[:-1] # delete last layer
        self.resnet152 = nn.Sequential(*modules)
        for param in self.resnet152.parameters():
            param.requires_grad = False
        
        self.fc1 = nn.Sequential(
            nn.Flatten(),
            nn.BatchNorm1d(resnet152.fc.in_features),
            nn.Dropout(0.4),
            nn.Linear(resnet152.fc.in_features, 256),
            nn.AdaptiveMaxPool1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, ntargets)
        )
    def forward(self, x):
        x = self.resnet152(x)
        x = self.fc1(x)
        return x

In [37]:
model_base_id = "microsoft/swin-base-patch4-window7-224-in22k"
#model_tiny_id = "microsoft/swin-tiny-patch4-window7-224"

class ActionClassifier_swin(nn.Module):
    def __init__(self, ntargets):
        super().__init__()
        swin = SwinForImageClassification.from_pretrained(model_base_id)
        modules = list(swin.children())[:-1] # delete last layer
        self.swin = nn.Sequential(*modules)
        for param in self.swin.parameters():
            param.requires_grad = False
        self.fc = nn.Sequential(
            nn.ReLU(),
            nn.Flatten(),
            nn.BatchNorm1d(1024), ##768 for tiny model
            nn.Dropout(0.2),
            nn.Linear(1024, ntargets)
        )
    
    def forward(self, x):
        output = self.swin(x)[:2]
        x = self.fc(output[1])
        return x

In [38]:
classifier_resnet152 = ActionClassifier_resnet152(len(ind2cat))
_ = summary(classifier_resnet152, torch.zeros(32,3,224,224).to(device))

In [39]:
classifier_swin = ActionClassifier_swin(len(ind2cat))
_ = summary(classifier_swin, torch.zeros(32,3,224,224).to(device))

## Train

In [40]:
def train(data, classifier_resnet152, classifier_swin, optimizer_VGG, optimizer_swin, loss_fn):
    imgs, targets = data
    classifier_resnet152.train()
    outputs_resnet152= classifier_resnet152(imgs)
    classifier_swin.train()
    outputs_swin = classifier_swin(imgs)
    outputs = ((outputs_resnet152 * 0.6) + (outputs_swin * 0.4)) / 2 
    loss = loss_fn(outputs, targets)
    preds = outputs.argmax(-1)
    acc = (sum(preds==targets) / len(targets))
    classifier_resnet152.zero_grad()
    classifier_swin.zero_grad()
    loss.backward()
    optimizer_resnet152.step()
    optimizer_swin.step()
    return loss, acc

In [41]:
@torch.no_grad()
def validate(data, classifier_resnet152,classifier_swin, loss_fn):
    imgs, targets = data
    classifier_resnet152.eval()
    outputs_resnet152 = classifier_resnet152(imgs)
    classifier_swin.eval()
    outputs_swin = classifier_swin(imgs)
    outputs = ((outputs_resnet152 * 0.6) + (outputs_swin * 0.4)) / 2  
    loss = loss_fn(outputs, targets)
    preds = outputs.argmax(-1)
    acc = (sum(preds==targets) / len(targets))
    return loss, acc

In [42]:
n_epochs = 20
log = Report(n_epochs)
classifier_resnet152 = ActionClassifier_resnet152(len(ind2cat)).to(device)
classifier_swin = ActionClassifier_swin(len(ind2cat)).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer_resnet152 = optim.Adam(classifier_resnet152.parameters())
scheduler_resnet152 = optim.lr_scheduler.StepLR(optimizer_resnet152, step_size=10,gamma=0.5)
optimizer_swin = optim.Adam(classifier_swin.parameters())
scheduler_swin = optim.lr_scheduler.StepLR(optimizer_swin, step_size=10,gamma=0.5)

In [43]:
for epoch in range(n_epochs):
    n_batch = len(train_dl)
    for i, data in enumerate(train_dl):
        train_loss, train_acc = train(data, classifier_resnet152, classifier_swin, 
                                      optimizer_resnet152, optimizer_swin, loss_fn)
        pos = epoch + ((i+1)/n_batch)
        log.record(pos=pos, train_loss=train_loss, 
                   train_acc=train_acc, end='\r')
        
    n_batch = len(val_dl)
    for i, data in enumerate(val_dl):
        val_loss, val_acc = validate(data, classifier_resnet152, classifier_swin, loss_fn)
        pos = epoch + ((i+1)/n_batch)
        log.record(pos=pos, val_loss=val_loss, val_acc=val_acc, 
                   end='\r')
    
    scheduler_resnet152.step()
    scheduler_swin.step()
    log.report_avgs(epoch+1)

In [44]:
log.plot_epochs(['train_loss', 'val_loss'])

In [45]:
log.plot_epochs(['train_acc', 'val_acc'])

In [46]:
!mkdir saved_models
torch.save(classifier_resnet152.state_dict(), './saved_models/classifier_weights_resnet50.pth')
torch.save(classifier_swin.state_dict(), './saved_models/classifier_weights_swin.pth')